# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data_file')
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
0,0,Rwamagana,RW,-1.95,30.43,62.67,67,100,1.90
1,1,Butaritari,KI,3.07,172.79,81.72,83,100,22.57
2,2,Rikitea,PF,-23.12,-134.97,78.94,77,100,15.75
3,3,Vestmannaeyjar,IS,63.44,-20.27,38.64,84,100,71.94
4,4,Bengkulu,ID,-3.80,102.27,88.56,54,17,1.66
...,...,...,...,...,...,...,...,...,...
556,556,Ahipara,NZ,-35.17,173.17,82.99,61,100,1.01
557,557,Anadyr,RU,64.75,177.48,8.60,78,20,8.95
558,558,Vila do Maio,CV,15.13,-23.22,77.00,36,33,11.41
559,559,Viransehir,TR,37.24,39.76,24.73,85,0,8.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

lat_long = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=300, point_radius=5 )
fig.add_layer(heat_layer)

In [4]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
0,0,Rwamagana,RW,-1.95,30.43,62.67,67,100,1.90
1,1,Butaritari,KI,3.07,172.79,81.72,83,100,22.57
2,2,Rikitea,PF,-23.12,-134.97,78.94,77,100,15.75
3,3,Vestmannaeyjar,IS,63.44,-20.27,38.64,84,100,71.94
4,4,Bengkulu,ID,-3.80,102.27,88.56,54,17,1.66


In [6]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


vacay_df = weather_df.loc[(weather_df['High Temp'] < 80 ) & (weather_df['High Temp'] > 70 ) 
                          & (weather_df['Wind Speed'] < 10 ) & (weather_df['Cloudiness'] == 0 )].dropna()



vacay_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
15,15,Jizan,SA,17.33,42.67,77.00,73,0,6.93
45,45,Siruguppa,IN,15.63,76.90,74.03,56,0,2.42
103,103,Ramtek,IN,21.40,79.33,71.60,49,0,2.24
130,130,Diglur,IN,18.55,77.60,73.11,35,0,2.35
144,144,Mae Chan,TH,20.15,99.85,79.00,47,0,3.36
173,173,Joao Pinheiro,BR,-17.74,-46.17,70.77,89,0,3.47
184,184,Champerico,GT,14.30,-91.92,77.14,79,0,5.08
212,212,Tirumullaivasal,IN,11.23,79.83,79.72,61,0,8.59
256,256,West Bay,KY,19.37,-81.42,79.00,88,0,9.17
263,263,Ballitoville,ZA,-29.54,31.21,75.99,88,0,1.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_df.copy()

hotel_df['Hotel Name'] = ""

hotel_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
15,15,Jizan,SA,17.33,42.67,77.00,73,0,6.93,
45,45,Siruguppa,IN,15.63,76.90,74.03,56,0,2.42,
103,103,Ramtek,IN,21.40,79.33,71.60,49,0,2.24,
130,130,Diglur,IN,18.55,77.60,73.11,35,0,2.35,
144,144,Mae Chan,TH,20.15,99.85,79.00,47,0,3.36,
173,173,Joao Pinheiro,BR,-17.74,-46.17,70.77,89,0,3.47,
184,184,Champerico,GT,14.30,-91.92,77.14,79,0,5.08,
212,212,Tirumullaivasal,IN,11.23,79.83,79.72,61,0,8.59,
256,256,West Bay,KY,19.37,-81.42,79.00,88,0,9.17,
263,263,Ballitoville,ZA,-29.54,31.21,75.99,88,0,1.12,


In [8]:
# geocoordinates

hotel_coords = []


params = {
   
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    hotel_coords.append((lat, long))
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    response_json = response.json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("None")
    



hotel_df


,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
15,15,Jizan,SA,17.33,42.67,77.00,73,0,6.93,شهدان. ابوالحجر
45,45,Siruguppa,IN,15.63,76.90,74.03,56,0,2.42,ಹಾಸ್ಟೆಲ್ ಸ್ಕೂಲ್ ಬಾಯ್ಸ್
103,103,Ramtek,IN,21.40,79.33,71.60,49,0,2.24,Kaveri Girls Hostel
130,130,Diglur,IN,18.55,77.60,73.11,35,0,2.35,Mirza Asad Baig (Residence)
144,144,Mae Chan,TH,20.15,99.85,79.00,47,0,3.36,Kiang Dao Resort
173,173,Joao Pinheiro,BR,-17.74,-46.17,70.77,89,0,3.47,Hotel Santos
184,184,Champerico,GT,14.30,-91.92,77.14,79,0,5.08,"Hotel y restaurante ""El Diamante"""
212,212,Tirumullaivasal,IN,11.23,79.83,79.72,61,0,8.59,Hajaali
256,256,West Bay,KY,19.37,-81.42,79.00,88,0,9.17,Calypso Cove
263,263,Ballitoville,ZA,-29.54,31.21,75.99,88,0,1.12,Hotel Fairmont Zimbali Resort


In [9]:
print(hotel_coords)

[(17.33, 42.67), (15.63, 76.9), (21.4, 79.33), (18.55, 77.6), (20.15, 99.85), (-17.74, -46.17), (14.3, -91.92), (11.23, 79.83), (19.37, -81.42), (-29.54, 31.21), (-5.09, -81.11), (13.08, 109.3), (23.57, 70.63), (21.28, -88.27), (19.22, -104.7), (7.62, -11.83), (4.92, 6.26)]


In [10]:
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.


# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_coords)
# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map